In [4]:
from Antara import Antara #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
obj = Antara()
end_date = date(2018, 8, 13)
start_date = date(2018, 2, 13)

In [6]:
init_page = 21
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%d-%m-%Y"))
    init_page = 1

2018-02-27
page  1
https://www.antaranews.com/search/%20/27-02-2018/27-02-2018/1
memasukkan berita id  689144
Insert berita  Arema raih tiket semifinal
masuk
memasukkan berita id  689143
Insert berita  KPK periksa delapan saksi kasus Bupati Jombang
masuk
memasukkan berita id  689142
Insert berita  Kemensos salurkan bantuan longsor Brebes Rp1,1 miliar
masuk
memasukkan berita id  689141
Insert berita  Dubes MIKTA untuk Takhta Suci Vatikan gelar seminar kontra-terorisme
masuk
memasukkan berita id  689140
Insert berita  Mahfud : &quot;sweeping&quot; itu melanggar undang-undang
masuk
memasukkan berita id  689139
Insert berita  Percasi mengharapkan catur bangkit lewat Novendra-Aditya
masuk
memasukkan berita id  689138
Insert berita  Legislator Bara Hasibuan ajak tolak paham radikal
masuk
memasukkan berita id  689137
Insert berita  Kapolda DIY: peristiwa penyerangan Gereja Santa Lidwina memprihatinkan
masuk
memasukkan berita id  689136
Insert berita  Banjir landa 30 kampung di Kabupaten Wayka

NameError: name 'sub' is not defined